<a href="https://colab.research.google.com/github/keisuke0502watanabe/DSC-pressure/blob/main/Roast_vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# グラフのスタイル設定
plt.style.use('default')  # seabornの代わりにデフォルトスタイルを使用
sns.set_theme()  # seabornのテーマを設定

# データの読み込み
file_path = '横田焙煎_202505161.csv'  # CSVファイルのパス

# 実験条件の読み込み（1-10行目）
conditions = pd.read_csv(file_path, nrows=9)
# 実験データの読み込み（11行目以降、11行目をヘッダーとして使用）
data = pd.read_csv(file_path, skiprows=10, header=0)

# メタデータの取得
roaster = conditions.iloc[0, 1]
date = conditions.iloc[1, 1]
bean = conditions.iloc[3, 1]
process = conditions.iloc[4, 1]

# 焙煎データの処理
roast_data = data.copy()

# 時間データの変換
def convert_time(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes + seconds/60

roast_data['Total time / min:sec'] = roast_data['Total time / min:sec'].apply(convert_time)
roast_data['Roast time / min:sec'] = roast_data['Roast time / min:sec'].apply(convert_time)

# 数値データの変換
numeric_columns = ['Bean Temp / C', 'Steam Temp / C', 'Gas pressure / kPa', 'Damper Level']
for col in numeric_columns:
    roast_data[col] = pd.to_numeric(roast_data[col], errors='coerce')



In [ ]:
data

,Process,Total time / min:sec,Roast time / min:sec,Bean Temp / C,Steam Temp / C,Gas pressure / kPa,Damper Level,Details
0,Preheat,0:00,0:00,143.5,171.5,2.0,3,NaN
1,Preheat,0:03,0:00,146.2,172.0,2.0,3,NaN
2,Preheat,0:05,0:00,148.0,172.8,2.0,3,NaN
3,Preheat,0:08,0:00,148.2,162.2,2.0,3,NaN
4,Preheat,0:10,0:00,146.0,148.5,2.0,3,NaN
...,...,...,...,...,...,...,...,...
345,Roast,14:39,12:29,エラー,エラー,1.8,1,NaN
346,Roast,14:41,12:32,エラー,エラー,1.8,1,NaN
347,Roast,14:43,12:34,エラー,エラー,1.8,1,NaN
348,Roast,14:45,12:36,エラー,エラー,1.8,1,NaN


In [ ]:
# prompt: dataの中身を修正したい：Preheatの時、Gas pressureを0にする

# 'Process' 列が 'Preheat' の行の 'Gas pressure / kPa' 列を 0 に設定
roast_data.loc[roast_data['Process'] == 'Preheat', 'Gas pressure / kPa'] = 0

# 修正されたデータフレームを表示して確認
print(roast_data.head())

   Process  Total time / min:sec  Roast time / min:sec  Bean Temp / C  \
0  Preheat              0.000000                   0.0          143.5   
1  Preheat              0.050000                   0.0          146.2   
2  Preheat              0.083333                   0.0          148.0   
3  Preheat              0.133333                   0.0          148.2   
4  Preheat              0.166667                   0.0          146.0   

   Steam Temp / C  Gas pressure / kPa  Damper Level Details  
0           171.5                 0.0             3     NaN  
1           172.0                 0.0             3     NaN  
2           172.8                 0.0             3     NaN  
3           162.2                 0.0             3     NaN  
4           148.5                 0.0             3     NaN  


In [ ]:
#このdataをcsvとして出力
roast_data.to_csv('roast_data-modifired-202505161.csv', index=False)

In [ ]:
# prompt: 文字化けするので英語表記で　視覚化plotlyで。横軸bean tempとsteam tempのグラフを一つ。gas pressrue のグラフ一つ。damperlebvel のグラフもさらに一つ。Detailsに'豆投入'時の時間をラベルする。

!pip install plotly

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: one for Bean Temp and Steam Temp, one for Gas Pressure, and one for Damper Level
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("Bean Temp and Steam Temp vs. Roast Time",
                                    "Gas Pressure vs. Roast Time",
                                    "Damper Level vs. Roast Time"))

# Add Bean Temp trace
fig.add_trace(go.Scattergl(x=roast_data['Roast time / min:sec'], y=roast_data['Bean Temp / C'],
                          mode='lines',
                          name='Bean Temp / C',
                          legendgroup='temp'),
              row=1, col=1)

# Add Steam Temp trace
fig.add_trace(go.Scattergl(x=roast_data['Roast time / min:sec'], y=roast_data['Steam Temp / C'],
                          mode='lines',
                          name='Steam Temp / C',
                          legendgroup='temp'),
              row=1, col=1)

# Add Gas Pressure trace
fig.add_trace(go.Scattergl(x=roast_data['Roast time / min:sec'], y=roast_data['Gas pressure / kPa'],
                          mode='lines',
                          name='Gas Pressure / kPa'),
              row=2, col=1)

# Add Damper Level trace
fig.add_trace(go.Scattergl(x=roast_data['Roast time / min:sec'], y=roast_data['Damper Level'],
                          mode='lines',
                          name='Damper Level'),
              row=3, col=1)

# Find the time when 'Event' is '豆投入' and add a vertical line and label
bean_in_time_row = roast_data[roast_data['Details'] == '豆投入']
if not bean_in_time_row.empty:
    bean_in_time = bean_in_time_row['Roast time / min:sec'].iloc[0]

    # Add vertical lines for 'Bean In'
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="Bean In", annotation_position="bottom left", row=1, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="Bean In", annotation_position="top left", row=2, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="Bean In", annotation_position="top left", row=3, col=1)

bean_in_time_row = roast_data[roast_data['Details'] == 'C1']
if not bean_in_time_row.empty:
    bean_in_time = bean_in_time_row['Roast time / min:sec'].iloc[0]

    # Add vertical lines for 'Bean In'
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="C1", annotation_position="bottom left", row=1, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="C1", annotation_position="top left", row=2, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="C1", annotation_position="top left", row=3, col=1)

bean_in_time_row = roast_data[roast_data['Details'] == 'End']
if not bean_in_time_row.empty:
    bean_in_time = bean_in_time_row['Roast time / min:sec'].iloc[0]

    # Add vertical lines for 'Bean In'
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="End", annotation_position="bottom left", row=1, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="End", annotation_position="top left", row=2, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="End", annotation_position="top left", row=3, col=1)



# Update layout
fig.update_layout(height=800, title_text=f"Roast Profile: {bean} ({date})")

# Update axis labels
fig.update_xaxes(title_text="Roast Time / min", row=1, col=1)
fig.update_yaxes(title_text="Temperature / C", row=1, col=1)
fig.update_xaxes(title_text="Roast Time / min", row=2, col=1)
fig.update_yaxes(title_text="Pressure / kPa", row=2, col=1)
fig.update_xaxes(title_text="Roast Time / min", row=3, col=1)
fig.update_yaxes(title_text="Level", row=3, col=1)

# Show the plot
fig2=fig
fig.show()


In [ ]:
# prompt: 文字化けするので英語表記で　視覚化plotlyで。横軸bean tempとsteam tempのグラフを一つ。gas pressrue のグラフ一つ。damperlebvel のグラフもさらに一つ。Detailsに'豆投入'時の時間をラベルする。

!pip install plotly

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: one for Bean Temp and Steam Temp, one for Gas Pressure, and one for Damper Level
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("Bean Temp and Steam Temp vs. Roast Time",
                                    "Gas Pressure vs. Roast Time",
                                    "Damper Level vs. Roast Time"))

# Add Bean Temp trace
fig.add_trace(go.Scattergl(x=roast_data['Total time / min:sec'], y=roast_data['Bean Temp / C'],
                          mode='lines',
                          name='Bean Temp / C',
                          legendgroup='temp'),
              row=1, col=1)

# Add Steam Temp trace
fig.add_trace(go.Scattergl(x=roast_data['Total time / min:sec'], y=roast_data['Steam Temp / C'],
                          mode='lines',
                          name='Steam Temp / C',
                          legendgroup='temp'),
              row=1, col=1)

# Add Gas Pressure trace
fig.add_trace(go.Scattergl(x=roast_data['Total time / min:sec'], y=roast_data['Gas pressure / kPa'],
                          mode='lines',
                          name='Gas Pressure / kPa'),
              row=2, col=1)

# Add Damper Level trace
fig.add_trace(go.Scattergl(x=roast_data['Total time / min:sec'], y=roast_data['Damper Level'],
                          mode='lines',
                          name='Damper Level'),
              row=3, col=1)

# Find the time when 'Event' is '豆投入' and add a vertical line and label
bean_in_time_row = roast_data[roast_data['Details'] == '豆投入']
if not bean_in_time_row.empty:
    bean_in_time = bean_in_time_row['Total time / min:sec'].iloc[0]

    # Add vertical lines for 'Bean In'
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="Bean In", annotation_position="bottom left", row=1, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="Bean In", annotation_position="top left", row=2, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="Bean In", annotation_position="top left", row=3, col=1)

bean_in_time_row = roast_data[roast_data['Details'] == 'C1']
if not bean_in_time_row.empty:
    bean_in_time = bean_in_time_row['Total time / min:sec'].iloc[0]

    # Add vertical lines for 'Bean In'
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="C1", annotation_position="bottom left", row=1, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="C1", annotation_position="top left", row=2, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="C1", annotation_position="top left", row=3, col=1)

bean_in_time_row = roast_data[roast_data['Details'] == 'End']
if not bean_in_time_row.empty:
    bean_in_time = bean_in_time_row['Total time / min:sec'].iloc[0]

    # Add vertical lines for 'Bean In'
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="End", annotation_position="bottom left", row=1, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="End", annotation_position="top left", row=2, col=1)
    fig.add_vline(x=bean_in_time, line_width=1, line_dash="dash", line_color="red",
                  annotation_text="End", annotation_position="top left", row=3, col=1)



# Update layout
fig.update_layout(height=800, title_text=f"Roast Profile: {bean} ({date})")

# Update axis labels
fig.update_xaxes(title_text="Total time / min:sec", row=1, col=1)
fig.update_yaxes(title_text="Temperature / C", row=1, col=1)
fig.update_xaxes(title_text="Total time / min:sec", row=2, col=1)
fig.update_yaxes(title_text="Pressure / kPa", row=2, col=1)
fig.update_xaxes(title_text="Total time / min:sec", row=3, col=1)
fig.update_yaxes(title_text="Level", row=3, col=1)

# Show the plot
fig.show()


In [ ]:
# 焙煎プロファイルの可視化
plt.figure(figsize=(12, 6))

# 豆温と蒸気温度のプロット
plt.plot(roast_data['Total time / min:sec'], roast_data['Bean Temp / C'],
         label='豆温', linewidth=2)
plt.plot(roast_data['Total time / min:sec'], roast_data['Steam Temp / C'],
         label='蒸気温度', linewidth=2, linestyle='--')

plt.title('{} - {} ({})\n焙煎日: {}'.format(roaster, bean, process, date), fontsize=14)
plt.xlabel('時間 (分)', fontsize=12)
plt.ylabel('温度 (°C)', fontsize=12)
plt.grid(True)
plt.legend(fontsize=10)

# ガス圧とダンパーレベルのプロット（2軸グラフ）
ax2 = plt.gca().twinx()
ax2.plot(roast_data['Total time / min:sec'], roast_data['Gas pressure / kPa'],
         label='ガス圧', color='green', linewidth=2)
ax2.plot(roast_data['Total time / min:sec'], roast_data['Damper Level'],
         label='ダンパーレベル', color='purple', linewidth=2, linestyle=':')
ax2.set_ylabel('ガス圧 (kPa) / ダンパーレベル', fontsize=12)
ax2.legend(loc='upper right', fontsize=10)

plt.tight_layout()
plt.show()